# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [4]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [5]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [6]:
# List all databases to confirm that uk_food exists
db_list = mongo.list_database_names()
print("Databases:", db_list)

Databases: ['admin', 'classDB', 'config', 'epa', 'fruits_db', 'local', 'met', 'petsitly_marketing', 'travel_db', 'uk_food']


In [7]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [9]:
# review the collections in our new database
collection_list = db.list_collection_names()
print("Collections in uk_food:", collection_list)

Collections in uk_food: ['establishments']


In [ ]:
# review a document in the establishments collection


In [10]:
# assign the collection to a variable
establishments = db['establishments']

In [11]:
document = establishments.find_one()

# Pretty-print the document to make it more readable
pprint(document)

{'AddressLine1': 'The Pines Garden',
 'AddressLine2': 'Beach Road',
 'AddressLine3': 'St Margarets Bay',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pines Calyx',
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4587.362402580997,
 'FHRSID': 254250,
 'LocalAuthorityBusinessID': 'PI/000066174',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DZ',
 'RatingDate': '2021-08-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('67326a5aadb464b3107df7e2'),
 'geocode': {'latitude': '51.148133', 'longitude': '1.383298'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254250',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extract

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [12]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "name": "Penang Flavours",
}

In [13]:
# Insert the new restaurant into the collection
result = establishments.insert_one(new_restaurant)

In [14]:
# Check that the new restaurant was inserted
print("New restaurant 'Penang Flavours' inserted with ID:", result.inserted_id)

New restaurant 'Penang Flavours' inserted with ID: 67326b8a25edf2f15b37f64f


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [15]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}

# Project only the BusinessTypeID and BusinessType fields
projection = {"BusinessTypeID": 1, "BusinessType": 1}

# Execute the query
result = establishments.find(query, projection)

# Print the results
for doc in result:
    pprint(doc)

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('67326a5aadb464b3107df7e4')}
{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('67326a5aadb464b3107df7e6')}
{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('67326a5aadb464b3107df7e7')}
{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('67326a5aadb464b3107df7e8')}
{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('67326a5aadb464b3107df7e9')}
{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('67326a5aadb464b3107df7eb')}
{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('67326a5aadb464b3107df7ec')}
{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('67326a5aadb464b3107df7ef')}
{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('67326a5aadb464b3107d

3. Update the new restaurant with the `BusinessTypeID` you found.

In [16]:
# Update the new restaurant with the correct BusinessTypeID
# Query to find the BusinessTypeID for "Restaurant/Cafe/Canteen"
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
projection = {"BusinessTypeID": 1}  # We only need the BusinessTypeID field

# Find the BusinessTypeID
business_type_doc = establishments.find_one(query, projection)

# Check if we found the BusinessTypeID
if business_type_doc:
    business_type_id = business_type_doc["BusinessTypeID"]
    
    # Query to update "Penang Flavours" restaurant with the correct BusinessTypeID
    update_query = {"name": "Penang Flavours"}  # Identify the restaurant to update
    update_data = {"$set": {"BusinessTypeID": business_type_id}}  # Set the BusinessTypeID
    
    # Perform the update
    establishments.update_one(update_query, update_data)
    
    print("Restaurant 'Penang Flavours' updated with BusinessTypeID:", business_type_id)
else:
    print("BusinessType 'Restaurant/Cafe/Canteen' not found.")

Restaurant 'Penang Flavours' updated with BusinessTypeID: 1


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [17]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName": "Dover"}

# Count the number of documents matching the query
count = establishments.count_documents(query)

# Print the result
print("Number of documents with LocalAuthorityName as 'Dover':", count)

Number of documents with LocalAuthorityName as 'Dover': 994


In [18]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {"LocalAuthorityName": "Dover"}

# Delete all documents matching the query
result = establishments.delete_many(query)

# Print the number of documents deleted
print(f"Number of documents deleted: {result.deleted_count}")

Number of documents deleted: 994


In [20]:
# Check if any remaining documents include Dover
query = {"LocalAuthorityName": "Dover"}

# Count the number of documents matching the query
count = establishments.count_documents(query)

# Print the result
print("Number of documents with LocalAuthorityName as 'Dover':", count)

Number of documents with LocalAuthorityName as 'Dover': 0


In [21]:
# Check that other documents remain with 'find_one'
remaining_doc = establishments.find_one()

# Check if a document was found and print it
if remaining_doc:
    print("A document still remains:", remaining_doc)
else:
    print("No documents remain.")

A document still remains: {'_id': ObjectId('67326a5aadb464b3107dfabc'), 'FHRSID': 1043695, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000073616', 'BusinessName': 'The Pavilion', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': 'East Cliff Pavilion', 'AddressLine2': 'Wear Bay Road', 'AddressLine3': 'Folkestone', 'AddressLine4': 'Kent', 'PostCode': 'CT19 6BL', 'Phone': '', 'RatingValue': '5', 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2018-04-04T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.195625', 'latitude': '51.083812'}, 'RightToReply': '', 'Distance': 4591.765489457773, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T0

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [22]:
# Change the data type from String to Decimal for longitude and latitude
from decimal import Decimal

# Query to match documents where 'longitude' and 'latitude' are stored as strings
query = {
    "longitude": {"$type": "string"},
    "latitude": {"$type": "string"}
}

# Update to convert the 'longitude' and 'latitude' fields to Decimal
update = {
    "$set": {
        "longitude": {"$toDecimal": "$longitude"},
        "latitude": {"$toDecimal": "$latitude"}
    }
}

# Apply the update to all matching documents
result = establishments.update_many(query, update)

# Print how many documents were updated
print(f"Number of documents updated: {result.modified_count}")


Number of documents updated: 0


Use `update_many` to convert `RatingValue` to integer numbers.

In [23]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [24]:
# Change the data type from String to Integer for RatingValue
query = {"RatingValue": {"$type": "string"}}

# Update to convert the 'RatingValue' field to Integer
update = {
    "$set": {
        "RatingValue": {"$toInt": "$RatingValue"}
    }
}

# Apply the update to all matching documents
result = establishments.update_many(query, update)

# Print how many documents were updated
print(f"Number of documents updated: {result.modified_count}")

Number of documents updated: 34694


In [25]:
# Check that the coordinates and rating value are now numbers
# Query to find a document that contains longitude, latitude, and RatingValue
query = {}

# Find one document that matches the query
document = establishments.find_one(query)

# Check and print the data types of the relevant fields
if document:
    print(f"Longitude: {document['longitude']} (Type: {type(document['longitude'])})")
    print(f"Latitude: {document['latitude']} (Type: {type(document['latitude'])})")
    print(f"RatingValue: {document['RatingValue']} (Type: {type(document['RatingValue'])})")
else:
    print("No documents found.")


KeyError: 'longitude'